# Iteration 8: bed occupancy (KPI) stage 2

## Plain English Summary

In this iteration we add a third basic performance measure calculation

4. Mean bed occupancy (utilization X number of beds)

These calculations only use statistics collected in the results collection period.#

We also make the printing of the simulated trace optional to reduce output when not needed in debugging.


## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## LLM Prompt

In [2]:
display(Markdown("../01_perplexity/inputs/iteration_8.md"))

modify the code as follows: 

Add in a new performance measure that is calculated at the end of the simulation run called "bed occupancy". This is calculated by multiplying the bed utilization by the number of critical care beds.

Add the result to the pandas dataframe.

add a new parameter to Experiment called "trace". The default value of "trace" is False.  If "trace" is True the model will print out simulated events such as admissions, discharges, or cancellations.   The model will always print out the performance at the end of the simulated run.

## Date and time of input to LLM

> 0925 6th June 2024 

## Observations on the LLM Output

* *Note from previous diff:* `utilisation` was removed from the CCU class. The mean wait time is not added to the CCU class and is calculated in the `run` function.

* bed occupancy calculation added in `run`. As utilisation is calculated as %, a modification is needed.

```python
bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds
```

* `trace` added as parameter to `Experiment` and printing of events is controlled via conditional logic e.g.

```python
    if self.experiment.trace:
        print(f"Patient {self.patient_id_counter} arrived from Accident and Emergency at {self.env.now:.2f} hours")
```
* NOTE: There is a bug in the code (picked up by tester):
  The unplanned admission and elective processes are outside the requests for a critical care bed.

## LLM Output 1 stage 2:

The changes made are:
Added a new parameter trace to the Experiment class with a default value of False. If trace is True, the model will print out simulated events such as admissions, discharges, or cancellations.
Added a new performance measure called "Bed Occupancy" calculated by multiplying the bed utilization by the number of critical care beds.
Added the "Bed Occupancy" result to the pandas dataframe.
Modified the accident_emergency_arrivals, unplanned_admissions_process, and elective_admissions_process methods to print out events if trace is True.
When running the simulation, set trace=True to enable event printing.
The model will always print out the performance measures at the end of the simulated run.

In [11]:
import simpy
import numpy as np
import pandas as pd
from scipy.stats import lognorm

class Experiment:
    def __init__(self, accident_emergency_arrival_rate=22.72, wards_arrival_rate=26.0, emergency_surgery_arrival_rate=37.0, other_hospitals_arrival_rate=47.2, xray_department_arrival_rate=575.0, elective_surgery_arrival_rate=17.91, elective_surgery_arrival_std_dev=3.16, accident_emergency_lognormal_mu=128.79, accident_emergency_lognormal_sigma=267.51, wards_lognormal_mu=177.89, wards_lognormal_sigma=276.54, emergency_surgery_lognormal_mu=140.15, emergency_surgery_lognormal_sigma=218.02, other_hospitals_lognormal_mu=212.86, other_hospitals_lognormal_sigma=457.67, xray_department_lognormal_mu=87.53, xray_department_lognormal_sigma=108.67, elective_surgery_treatment_rate=57.34, num_critical_care_beds=24, intensive_cleaning_duration=5, results_collection_period=12 * 30 * 24, warm_up_period=30 * 24, trace=False):
        self.accident_emergency_arrival_rate = accident_emergency_arrival_rate
        self.wards_arrival_rate = wards_arrival_rate
        self.emergency_surgery_arrival_rate = emergency_surgery_arrival_rate
        self.other_hospitals_arrival_rate = other_hospitals_arrival_rate
        self.xray_department_arrival_rate = xray_department_arrival_rate
        self.elective_surgery_arrival_rate = elective_surgery_arrival_rate
        self.elective_surgery_arrival_std_dev = elective_surgery_arrival_std_dev
        self.accident_emergency_lognormal_mu = accident_emergency_lognormal_mu
        self.accident_emergency_lognormal_sigma = accident_emergency_lognormal_sigma
        self.wards_lognormal_mu = wards_lognormal_mu
        self.wards_lognormal_sigma = wards_lognormal_sigma
        self.emergency_surgery_lognormal_mu = emergency_surgery_lognormal_mu
        self.emergency_surgery_lognormal_sigma = emergency_surgery_lognormal_sigma
        self.other_hospitals_lognormal_mu = other_hospitals_lognormal_mu
        self.other_hospitals_lognormal_sigma = other_hospitals_lognormal_sigma
        self.xray_department_lognormal_mu = xray_department_lognormal_mu
        self.xray_department_lognormal_sigma = xray_department_lognormal_sigma
        self.elective_surgery_treatment_rate = elective_surgery_treatment_rate
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.results_collection_period = results_collection_period
        self.warm_up_period = warm_up_period
        self.total_run_length = self.warm_up_period + self.results_collection_period
        self.trace = trace

class CCU:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_id_counter = 0
        self.cancelled_operations = 0
        self.warmup_end_time = self.experiment.warm_up_period
        self.total_unplanned_waiting_time = 0 # Initialize total waiting time
        self.total_unplanned_admissions = 0 # Initialize total unplanned admissions
        self.total_treatment_time = 0 # Initialize total treatment time
        
        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)
        
        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=self.experiment.num_critical_care_beds)

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(self.experiment.accident_emergency_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(self.experiment.wards_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(self.experiment.emergency_surgery_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(self.experiment.other_hospitals_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(self.experiment.xray_department_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_elective_surgery.normal(self.experiment.elective_surgery_arrival_rate, self.experiment.elective_surgery_arrival_std_dev))
            if self.experiment.trace:
                print(f"Elective Patient {self.patient_id_counter} arrived at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.elective_admissions_process())

    def unplanned_admissions_process(self, source):
        patient_id = self.patient_id_counter
        if self.experiment.trace:
            print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        waiting_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
        waiting_time = self.env.now - waiting_time
        if self.experiment.trace:
            print(f"Patient {patient_id} from {source} waited {waiting_time:.2f} hours")
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")
        
        # Determine length of stay based on source
        if source == "Accident and Emergency":
            mu, sigma = self.lognormal_to_normal(self.experiment.accident_emergency_lognormal_mu, self.experiment.accident_emergency_lognormal_sigma)
            length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
        elif source == "Wards":
            mu, sigma = self.lognormal_to_normal(self.experiment.wards_lognormal_mu, self.experiment.wards_lognormal_sigma)
            length_of_stay = self.rng_wards.lognormal(mu, sigma)
        elif source == "Emergency Surgery":
            mu, sigma = self.lognormal_to_normal(self.experiment.emergency_surgery_lognormal_mu, self.experiment.emergency_surgery_lognormal_sigma)
            length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
        elif source == "Other Hospitals":
            mu, sigma = self.lognormal_to_normal(self.experiment.other_hospitals_lognormal_mu, self.experiment.other_hospitals_lognormal_sigma)
            length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
        else: # X-Ray Department
            mu, sigma = self.lognormal_to_normal(self.experiment.xray_department_lognormal_mu, self.experiment.xray_department_lognormal_sigma)
            length_of_stay = self.rng_xray_department.lognormal(mu, sigma)
        
        yield self.env.timeout(length_of_stay)
        
        # Add treatment time and waiting time to totals if simulation time has exceeded warm-up period
        if self.env.now >= self.warmup_end_time:
            self.total_treatment_time += length_of_stay
            self.total_unplanned_waiting_time += waiting_time
            self.total_unplanned_admissions += 1
        
        if self.experiment.trace:
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")
        
        # Intensive cleaning
        yield self.env.timeout(self.experiment.intensive_cleaning_duration)
        if self.experiment.trace:
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self):
        patient_id = self.patient_id_counter
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            if self.env.now >= self.warmup_end_time:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
                self.cancelled_operations += 1
            else:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds (warm-up period)")
        else:
            if self.experiment.trace:
                print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
            with self.critical_care_beds.request() as req:
                yield req
            if self.experiment.trace:
                print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
            length_of_stay = self.rng_elective_surgery.exponential(self.experiment.elective_surgery_treatment_rate)
            yield self.env.timeout(length_of_stay)
            # Add treatment time to total if simulation time has exceeded warm-up period
            if self.env.now >= self.warmup_end_time:
                self.total_treatment_time += length_of_stay
            if self.experiment.trace:
                print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")
            # Intensive cleaning
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())

        self.env.run(until=self.experiment.total_run_length)

        # Calculate bed utilization
        bed_utilization = (self.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)) * 100

        # Calculate bed occupancy
        bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds

        # Calculate performance measures
        performance_measures = {
            "Total Cancelled Elective Operations": self.cancelled_operations,
            "Mean Unplanned Admission Waiting Time (hours)": self.total_unplanned_waiting_time / self.total_unplanned_admissions if self.total_unplanned_admissions > 0 else 0,
            "Bed Utilization (%)": bed_utilization,
            "Bed Occupancy": bed_occupancy
        }

        # Output results in a pandas dataframe
        results_df = pd.DataFrame.from_dict(performance_measures, orient='index', columns=['Value'])
        print(results_df)

# Run the simulation
experiment = Experiment(trace=False)  # Set trace=True to enable event printing
env = simpy.Environment()
ccu = CCU(env, experiment)
ccu.run()


                                                     Value
Total Cancelled Elective Operations               0.000000
Mean Unplanned Admission Waiting Time (hours)     0.000000
Bed Utilization (%)                             107.525913
Bed Occupancy                                  2580.621921


## Additional prompt: 
Modify the modified code as follows: all activities in the unplanned_admission process and elective_admissions_process require a critical care bed and need to be moved inside the critical care bed request block.

In [12]:
import simpy
import numpy as np
import pandas as pd
from scipy.stats import lognorm

class Experiment:
    def __init__(self, accident_emergency_arrival_rate=22.72, wards_arrival_rate=26.0, emergency_surgery_arrival_rate=37.0, other_hospitals_arrival_rate=47.2, xray_department_arrival_rate=575.0, elective_surgery_arrival_rate=17.91, elective_surgery_arrival_std_dev=3.16, accident_emergency_lognormal_mu=128.79, accident_emergency_lognormal_sigma=267.51, wards_lognormal_mu=177.89, wards_lognormal_sigma=276.54, emergency_surgery_lognormal_mu=140.15, emergency_surgery_lognormal_sigma=218.02, other_hospitals_lognormal_mu=212.86, other_hospitals_lognormal_sigma=457.67, xray_department_lognormal_mu=87.53, xray_department_lognormal_sigma=108.67, elective_surgery_treatment_rate=57.34, num_critical_care_beds=24, intensive_cleaning_duration=5, results_collection_period=12 * 30 * 24, warm_up_period=30 * 24, trace=False):
        self.accident_emergency_arrival_rate = accident_emergency_arrival_rate
        self.wards_arrival_rate = wards_arrival_rate
        self.emergency_surgery_arrival_rate = emergency_surgery_arrival_rate
        self.other_hospitals_arrival_rate = other_hospitals_arrival_rate
        self.xray_department_arrival_rate = xray_department_arrival_rate
        self.elective_surgery_arrival_rate = elective_surgery_arrival_rate
        self.elective_surgery_arrival_std_dev = elective_surgery_arrival_std_dev
        self.accident_emergency_lognormal_mu = accident_emergency_lognormal_mu
        self.accident_emergency_lognormal_sigma = accident_emergency_lognormal_sigma
        self.wards_lognormal_mu = wards_lognormal_mu
        self.wards_lognormal_sigma = wards_lognormal_sigma
        self.emergency_surgery_lognormal_mu = emergency_surgery_lognormal_mu
        self.emergency_surgery_lognormal_sigma = emergency_surgery_lognormal_sigma
        self.other_hospitals_lognormal_mu = other_hospitals_lognormal_mu
        self.other_hospitals_lognormal_sigma = other_hospitals_lognormal_sigma
        self.xray_department_lognormal_mu = xray_department_lognormal_mu
        self.xray_department_lognormal_sigma = xray_department_lognormal_sigma
        self.elective_surgery_treatment_rate = elective_surgery_treatment_rate
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.results_collection_period = results_collection_period
        self.warm_up_period = warm_up_period
        self.total_run_length = self.warm_up_period + self.results_collection_period
        self.trace = trace

class CCU:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_id_counter = 0
        self.cancelled_operations = 0
        self.warmup_end_time = self.experiment.warm_up_period
        self.total_unplanned_waiting_time = 0 # Initialize total waiting time
        self.total_unplanned_admissions = 0 # Initialize total unplanned admissions
        self.total_treatment_time = 0 # Initialize total treatment time
        
        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)
        
        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=self.experiment.num_critical_care_beds)

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(self.experiment.accident_emergency_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(self.experiment.wards_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(self.experiment.emergency_surgery_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(self.experiment.other_hospitals_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(self.experiment.xray_department_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_elective_surgery.normal(self.experiment.elective_surgery_arrival_rate, self.experiment.elective_surgery_arrival_std_dev))
            if self.experiment.trace:
                print(f"Elective Patient {self.patient_id_counter} arrived at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.elective_admissions_process())

    def unplanned_admissions_process(self, source):
        patient_id = self.patient_id_counter
        if self.experiment.trace:
            print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        waiting_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
            waiting_time = self.env.now - waiting_time
            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} waited {waiting_time:.2f} hours")
                print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.experiment.accident_emergency_lognormal_mu, self.experiment.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.experiment.wards_lognormal_mu, self.experiment.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.experiment.emergency_surgery_lognormal_mu, self.experiment.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.experiment.other_hospitals_lognormal_mu, self.experiment.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else: # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.experiment.xray_department_lognormal_mu, self.experiment.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)

            # Add treatment time and waiting time to totals if simulation time has exceeded warm-up period
            if self.env.now >= self.warmup_end_time:
                self.total_treatment_time += length_of_stay
                self.total_unplanned_waiting_time += waiting_time
                self.total_unplanned_admissions += 1

            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} requests a critical care bed for intensive cleaning at {self.env.now:.2f} hours")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self):
        patient_id = self.patient_id_counter
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            if self.env.now >= self.warmup_end_time:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
                self.cancelled_operations += 1
            else:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds (warm-up period)")
        else:
            if self.experiment.trace:
                print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
            with self.critical_care_beds.request() as req:
                yield req
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(self.experiment.elective_surgery_treatment_rate)
                yield self.env.timeout(length_of_stay)
                # Add treatment time to total if simulation time has exceeded warm-up period
                if self.env.now >= self.warmup_end_time:
                    self.total_treatment_time += length_of_stay
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")
                # Intensive cleaning
                yield self.env.timeout(self.experiment.intensive_cleaning_duration)
                if self.experiment.trace:
                    print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())

        self.env.run(until=self.experiment.total_run_length)

        # Calculate bed utilization
        bed_utilization = (self.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)) * 100

        # Calculate bed occupancy
        bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds

        # Calculate performance measures
        performance_measures = {
            "Total Cancelled Elective Operations": self.cancelled_operations,
            "Mean Unplanned Admission Waiting Time (hours)": self.total_unplanned_waiting_time / self.total_unplanned_admissions if self.total_unplanned_admissions > 0 else 0,
            "Bed Utilization (%)": bed_utilization,
            "Bed Occupancy": bed_occupancy
        }

        # Output results in a pandas dataframe
        results_df = pd.DataFrame.from_dict(performance_measures, orient='index', columns=['Value'])
        print(results_df)

# Run the simulation
experiment = Experiment(trace=False)  # Set trace=True to enable event printing
env = simpy.Environment()
ccu = CCU(env, experiment)
ccu.run()


                                                     Value
Total Cancelled Elective Operations             324.000000
Mean Unplanned Admission Waiting Time (hours)    24.652835
Bed Utilization (%)                              88.427355
Bed Occupancy                                  2122.256528


## Additional prompt:
Modify the calculation of utilisation so that it is expressed as a proportion, rather than a percentage.

## LLM output:

In this modified code, the line bed_utilization = self.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period) calculates the bed utilization as a proportion between 0 and 1.
The bed occupancy calculation remains the same: bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds.
The performance measures dictionary now includes "Bed Utilization" as a proportion instead of a percentage.

In [13]:
import simpy
import numpy as np
import pandas as pd
from scipy.stats import lognorm

class Experiment:
    def __init__(self, accident_emergency_arrival_rate=22.72, wards_arrival_rate=26.0, emergency_surgery_arrival_rate=37.0, other_hospitals_arrival_rate=47.2, xray_department_arrival_rate=575.0, elective_surgery_arrival_rate=17.91, elective_surgery_arrival_std_dev=3.16, accident_emergency_lognormal_mu=128.79, accident_emergency_lognormal_sigma=267.51, wards_lognormal_mu=177.89, wards_lognormal_sigma=276.54, emergency_surgery_lognormal_mu=140.15, emergency_surgery_lognormal_sigma=218.02, other_hospitals_lognormal_mu=212.86, other_hospitals_lognormal_sigma=457.67, xray_department_lognormal_mu=87.53, xray_department_lognormal_sigma=108.67, elective_surgery_treatment_rate=57.34, num_critical_care_beds=24, intensive_cleaning_duration=5, results_collection_period=12 * 30 * 24, warm_up_period=30 * 24, trace=False):
        self.accident_emergency_arrival_rate = accident_emergency_arrival_rate
        self.wards_arrival_rate = wards_arrival_rate
        self.emergency_surgery_arrival_rate = emergency_surgery_arrival_rate
        self.other_hospitals_arrival_rate = other_hospitals_arrival_rate
        self.xray_department_arrival_rate = xray_department_arrival_rate
        self.elective_surgery_arrival_rate = elective_surgery_arrival_rate
        self.elective_surgery_arrival_std_dev = elective_surgery_arrival_std_dev
        self.accident_emergency_lognormal_mu = accident_emergency_lognormal_mu
        self.accident_emergency_lognormal_sigma = accident_emergency_lognormal_sigma
        self.wards_lognormal_mu = wards_lognormal_mu
        self.wards_lognormal_sigma = wards_lognormal_sigma
        self.emergency_surgery_lognormal_mu = emergency_surgery_lognormal_mu
        self.emergency_surgery_lognormal_sigma = emergency_surgery_lognormal_sigma
        self.other_hospitals_lognormal_mu = other_hospitals_lognormal_mu
        self.other_hospitals_lognormal_sigma = other_hospitals_lognormal_sigma
        self.xray_department_lognormal_mu = xray_department_lognormal_mu
        self.xray_department_lognormal_sigma = xray_department_lognormal_sigma
        self.elective_surgery_treatment_rate = elective_surgery_treatment_rate
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.results_collection_period = results_collection_period
        self.warm_up_period = warm_up_period
        self.total_run_length = self.warm_up_period + self.results_collection_period
        self.trace = trace

class CCU:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_id_counter = 0
        self.cancelled_operations = 0
        self.warmup_end_time = self.experiment.warm_up_period
        self.total_unplanned_waiting_time = 0 # Initialize total waiting time
        self.total_unplanned_admissions = 0 # Initialize total unplanned admissions
        self.total_treatment_time = 0 # Initialize total treatment time
        
        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)
        
        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=self.experiment.num_critical_care_beds)

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(self.experiment.accident_emergency_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(self.experiment.wards_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(self.experiment.emergency_surgery_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(self.experiment.other_hospitals_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(self.experiment.xray_department_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_elective_surgery.normal(self.experiment.elective_surgery_arrival_rate, self.experiment.elective_surgery_arrival_std_dev))
            if self.experiment.trace:
                print(f"Elective Patient {self.patient_id_counter} arrived at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.elective_admissions_process())

    def unplanned_admissions_process(self, source):
        patient_id = self.patient_id_counter
        if self.experiment.trace:
            print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        waiting_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
            waiting_time = self.env.now - waiting_time
            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} waited {waiting_time:.2f} hours")
                print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.experiment.accident_emergency_lognormal_mu, self.experiment.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.experiment.wards_lognormal_mu, self.experiment.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.experiment.emergency_surgery_lognormal_mu, self.experiment.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.experiment.other_hospitals_lognormal_mu, self.experiment.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else: # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.experiment.xray_department_lognormal_mu, self.experiment.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)

            # Add treatment time and waiting time to totals if simulation time has exceeded warm-up period
            if self.env.now >= self.warmup_end_time:
                self.total_treatment_time += length_of_stay
                self.total_unplanned_waiting_time += waiting_time
                self.total_unplanned_admissions += 1

            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} requests a critical care bed for intensive cleaning at {self.env.now:.2f} hours")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self):
        patient_id = self.patient_id_counter
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            if self.env.now >= self.warmup_end_time:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
                self.cancelled_operations += 1
            else:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds (warm-up period)")
        else:
            if self.experiment.trace:
                print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
            with self.critical_care_beds.request() as req:
                yield req
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(self.experiment.elective_surgery_treatment_rate)
                yield self.env.timeout(length_of_stay)
                # Add treatment time to total if simulation time has exceeded warm-up period
                if self.env.now >= self.warmup_end_time:
                    self.total_treatment_time += length_of_stay
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")
                # Intensive cleaning
                yield self.env.timeout(self.experiment.intensive_cleaning_duration)
                if self.experiment.trace:
                    print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())

        self.env.run(until=self.experiment.total_run_length)

        # Calculate bed utilization
        bed_utilization = self.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)

        # Calculate bed occupancy
        bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds

        # Calculate performance measures
        performance_measures = {
            "Total Cancelled Elective Operations": self.cancelled_operations,
            "Mean Unplanned Admission Waiting Time (hours)": self.total_unplanned_waiting_time / self.total_unplanned_admissions if self.total_unplanned_admissions > 0 else 0,
            "Bed Utilization": bed_utilization,
            "Bed Occupancy": bed_occupancy
        }

        # Output results in a pandas dataframe
        results_df = pd.DataFrame.from_dict(performance_measures, orient='index', columns=['Value'])
        print(results_df)

# Run the simulation
experiment = Experiment(trace=False)  # Set trace=True to enable event printing
env = simpy.Environment()
ccu = CCU(env, experiment)
ccu.run()


                                                    Value
Total Cancelled Elective Operations            324.000000
Mean Unplanned Admission Waiting Time (hours)   24.652835
Bed Utilization                                  0.884274
Bed Occupancy                                   21.222565


## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [14]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [15]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCU(env, experiment)
    ccu_model.run()
    return ccu_model

In [16]:
def extreme_value_test_1(extreme_value=M*5):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time
    set to $M$ a very large number
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    experiment = Experiment(wards_arrival_rate = extreme_value,
                            emergency_surgery_arrival_rate = extreme_value, other_hospitals_arrival_rate = extreme_value,
                            xray_department_arrival_rate = extreme_value, elective_surgery_arrival_rate = extreme_value)
    run_test(experiment)

In [7]:
extreme_value_test_1()

   Cancelled Elective Operations  Bed Utilization  \
0                              0         0.216458   

   Mean Waiting Time Unplanned  Bed Occupancy  
0                          0.0       5.194989  


### Test 2: Extreme value test 2

PASS

In [17]:
def extreme_value_test_2(extreme_value=M*5):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    experiment = Experiment(accident_emergency_arrival_rate = extreme_value, wards_arrival_rate = extreme_value,
                            emergency_surgery_arrival_rate = extreme_value, other_hospitals_arrival_rate = extreme_value,
                            xray_department_arrival_rate = extreme_value, trace=True)
    run_test(experiment)

In [18]:
try:
    extreme_value_test_2()
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

Elective Patient 0 arrived at 15.80 hours
Elective Patient 1 requests a critical care bed at 15.80 hours
Elective Patient 1 admitted to a critical care bed at 15.80 hours
Elective Patient 1 arrived at 34.02 hours
Elective Patient 2 requests a critical care bed at 34.02 hours
Elective Patient 2 admitted to a critical care bed at 34.02 hours
Elective Patient 2 arrived at 45.80 hours
Elective Patient 3 requests a critical care bed at 45.80 hours
Elective Patient 3 admitted to a critical care bed at 45.80 hours
Elective Patient 3 arrived at 61.18 hours
Elective Patient 4 requests a critical care bed at 61.18 hours
Elective Patient 4 admitted to a critical care bed at 61.18 hours
Elective Patient 3 discharged from a critical care bed at 61.98 hours
Intensive cleaning completed for Elective Patient 3 at 66.98 hours
Elective Patient 4 arrived at 77.65 hours
Elective Patient 5 requests a critical care bed at 77.65 hours
Elective Patient 5 admitted to a critical care bed at 77.65 hours
Elective

### Test 3: Different run length

PASS

In [19]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`

    Note: metrics not collected as new_collection_period is less than warm-up period.
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [20]:
test_results_collection_period(10)

                                               Value
Total Cancelled Elective Operations              0.0
Mean Unplanned Admission Waiting Time (hours)    0.0
Bed Utilization                                  0.0
Bed Occupancy                                    0.0


730

### Test 4: Extreme value test 3

PASS.

In [23]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.

    Note: reported utilisation will not quite be 100% because of the patient still
    in service.
    
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    ccu_model = run_test(experiment)
    print(f'{ccu_model.total_treatment_time=}')
    print(f'{experiment.results_collection_period=}')

In [24]:
extreme_value_test_3()

                                                     Value
Total Cancelled Elective Operations             482.000000
Mean Unplanned Admission Waiting Time (hours)  4904.876563
Bed Utilization                                   0.960624
Bed Occupancy                                     0.960624
ccu_model.total_treatment_time=8299.793648035462
experiment.results_collection_period=8640


### Test 5: Extreme value test 4

PASS

In [27]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.

    Note: as patients do not leave beds, wait time, utilisation, and occupancy are not calculated.
    '''
    experiment = Experiment(accident_emergency_lognormal_mu=M, 
                 wards_lognormal_mu=M, 
                 emergency_surgery_lognormal_mu=M, 
                 other_hospitals_lognormal_mu=M, 
                 xray_department_lognormal_mu=M, 
                 elective_surgery_treatment_rate=M,
                 trace=True)
    run_test(experiment)

In [28]:
try:
    extreme_value_test_4()
except ZeroDivisionError:
    print("caught expected zero division error due to extreme value test")

Patient 0 arrived from the Wards at 4.30 hours
Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards waited 0.00 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 from Wards requests a critical care bed at 6.54 hours
Patient 2 from Wards waited 0.00 hours
Patient 2 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 arrived from Emergency Surgery at 13.71 hours
Patient 3 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 3 from Emergency Surgery waited 0.00 hours
Patient 3 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Elective Patient 3 arrived at 15.80 hours
Elective Patient 4 requests a critical care bed at 15.80 hours
Elective Patient 4 admitted to a critical care bed at 15.80 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 from Emergency Surgery requests a critical care bed at 31.66 hour

### Test 6: Vary Warm-up parameter

PASS

In [29]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [30]:
test_warmup_period(0)

                                                    Value
Total Cancelled Elective Operations            288.000000
Mean Unplanned Admission Waiting Time (hours)   22.374296
Bed Utilization                                  0.835263
Bed Occupancy                                   20.046319


(8640, 8640)